## _*Using Qiskit Aqua for exact cover problems*_

In mathematics, given a collection $S$ of subsets of a set $X$.
An exact cover is a subcollection $S_{ec} \subseteq S$ such that each element in $X$ is contained in exactly one subset $\in S_{ec}$. 

We will go through two examples to show:
1. How to run the optimization
2. How how to run the optimization with the VQE.

#### The problem and the brute-force method.

First, let us take a look at the list of subsets.

In [1]:
import numpy as np
import json

from qiskit import BasicAer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.applications.ising import exact_cover
from qiskit.optimization.applications.ising.common import sample_most_likely

input_file = 'sample.exactcover'
with open(input_file) as f:
    list_of_subsets = json.load(f)
    print(list_of_subsets)

[[2, 3, 4], [1, 2], [3, 4], [1, 2, 3]]


Then we apply the brute-force method. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a subset is either 0 (meaning the subset is not in the cover) or 1 (meaning the subset is in the cover). We print the binary assignment that satisfies the definition of the exact cover.

In [2]:
def brute_force():
    # brute-force way: try every possible assignment!
    has_sol = False

    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = len(list_of_subsets)
    max = 2**L
    for i in range(max):
        cur = bitfield(i, L)
        cur_v = exact_cover.check_solution_satisfiability(cur, list_of_subsets)
        if cur_v:
            has_sol = True
            break
    return has_sol, cur

has_sol, cur = brute_force()
if has_sol:
    print("Solution is", cur)
else:
    print("No solution is found")

Solution is [0, 1, 1, 0]


In [3]:
qubit_op, offset = exact_cover.get_operator(list_of_subsets)

#### Part I: Run the optimization

In [4]:
algo = NumPyMinimumEigensolver(qubit_op, aux_operators=[])
result = algo.run()

x = sample_most_likely(result.eigenstate)
ising_sol = exact_cover.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1, 0])

if exact_cover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("Solution is", ising_sol)
else:
    print("No solution is found")

Solution is [0 1 1 0]


#### Part II: Run the optimization with the VQE

In [5]:
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal

aqua_globals.random_seed = 10598

optimizer = COBYLA()
var_form = TwoLocal(qubit_op.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
vqe = VQE(qubit_op, var_form, optimizer)

backend = BasicAer.get_backend('statevector_simulator')
result = vqe.run(backend)

x = sample_most_likely(result.eigenstate)
ising_sol = exact_cover.get_solution(x)
if exact_cover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("Solution is", ising_sol)
else:
    print("No solution is found")

Solution is [0. 1. 1. 0.]
